In [1]:
import os
import glob
import traceback
from datetime import datetime
import pandas as pd
from plotly.graph_objs import Scatter,Layout
import plotly
import plotly.offline as py
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
plotly.offline.init_notebook_mode(connected=True)
import gezi
from gezi import tqdm, line
import pymp
from multiprocessing import Pool, Manager, cpu_count 
import qgrid

In [2]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:50000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [3]:
mark = 'tuwen'
root = f'/home/gezi/tmp/rank/exps/tuwen/v26'
root = f'/home/gezi/tmp/rank/exps/tuwen/v27'
# root = f'/home/gezi/tmp/rank/exps/{mark}/monitor'
base_dir = f'/search/odin/publicData/CloudS/libowei/rank_online/infos/{mark}/8'
# base_dir = f'/search/odin/publicData/CloudS/rank/infos/{mark}/16.new'

In [4]:
def gen_df(mark):
  dfs_offline = Manager().list()
  dfs_online = Manager().list()
  files = glob.glob(f'{base_dir}/*/sgsapp_metrics_offline_impression.csv')
#   files = sorted(files, key=lambda x: os.path.getmtime(x))[:100]
  ps = min(len(files), cpu_count())
  with pymp.Parallel(ps) as p:
    for i in tqdm(p.range(len(files)),desc='offline'):
      file = files[i]
      if not gezi.non_empty(file):
        continue
      df = pd.read_csv(file)
      dfs_offline.append(df)
  files = glob.glob(f'{base_dir}/*/sgsapp_metrics_online_impression.csv')
  files = sorted(files, key=lambda x: os.path.getmtime(x))[:100]
  ps = min(len(files), cpu_count())
  with pymp.Parallel(ps) as p:
    for i in tqdm(p.range(len(files)), desc='online'):
      file = files[i]
      if not gezi.non_empty(file):
        continue
      df = pd.read_csv(file)
      dfs_online.append(df)
  df_off = pd.concat(list(dfs_offline))
  df_on = pd.concat(list(dfs_online))
  return df_off, df_on

In [ ]:
df_base_off, df_base_on = gen_df(mark)
df_base_off = df_base_off[df_base_off.abtest==45600]
df_base_off = df_base_off.groupby(df_base_off.hour, as_index=False).last()
# df_base_on = df_base_on[df_base_on.abtest==456]
# df_base_on = df_base_on.groupby(df_base_on.hour, as_index=False).last()
# df_base_off['hour'] = df_base_off['hour'].astype(int)
# df_base_on['hour'] = df_base_on['hour'].astype(int)

offline: 100%|██████████| 19/19 [00:03<00:00,  4.37it/s]

offline: 100%|██████████| 19/19 [00:03<00:00,  5.50it/s]


offline:  83%|████████▎ | 15/18 [00:03<00:00,  4.18it/s]

In [ ]:
df_base_off.hour

In [ ]:
dfs = []
m = {}
start_hour = 1e32
end_hour = -1
base_records = []
for dir in tqdm(glob.glob(f'{root}/*')):
  if os.path.isdir(dir):
    try:
      df = pd.read_csv(f'{dir}/metric_hours.csv')
      df = df[~df.hour.isnull()]
      df['hour'] = df['hour'].astype(int)
      df = df.groupby(df.hour, as_index=False).last().sort_values(by=['hour'])
      df = df.replace(0., np.NaN)
      name = os.path.basename(dir)
      name = name.replace('deeponly-', '')
      df['model'] = name
      df.name = name
      m[name] = df
      if 'timestamp' in df.columns:
        timestamps = df['timestamp'].values
        df['duration'] = [0, *((timestamps[1:] - timestamps[:-1]) / 60)]
      if len(df):
        dfs.append(df)
      else:
        print('empty df:', f'{dir}/metric_hours.csv')
      if df.hour.max() > end_hour:
        end_hour = df.hour.max()
      if df.hour.min() < start_hour:
        start_hour = df.hour.min()
      try:
        df_record = pd.read_csv(f'{dir}/base_metric_hours.csv')
        df_record = df_record.replace(0., np.NaN)
        base_records.append(df_record)
      except Exception:
        pass
    except Exception:
#       print(traceback.format_exc())
      continue
         
if len(base_records):
  df_base_records = pd.concat(base_records)
  df_base_records = df_base_records.groupby(df_base_records.hour, as_index=False).last().sort_values(by=['hour'])
  name = 'base'
  df_base_records['model'] = name
  df_base_records.name = name
  dfs.append(df_base_records)
  m[name] = df_base_records
    
df_base_off['hour'] = df_base_off['hour'].astype(int)
# df_base_on['hour'] = df_base_on['hour'].astype(int)

# df_base_off_ = df_base_off[df_base_off.hour >= int(start_hour)][df_base_off.hour <= int(end_hour)]

# df_base_ = df_base[df_base.hour >= start_hour]
if len(df_base_off):
  name = 'baseline.off'
  df_base_off['model'] = name
  df_base_off.name = name
  dfs.append(df_base_off)
  m[name] = df_base_off
  
# df_base_on_ = df_base_on[df_base_on.hour >= int(start_hour)][df_base_on.hour <= int(end_hour)]
# # df_base_ = df_base[df_base.hour >= start_hour]
# if len(df_base_on_):
#   name = 'baseline.on'
#   df_base_on['model'] = name
#   df_base_on.name = name
#   dfs.append(df_base_on)
#   m[name] = df_base_on

for i in range(len(dfs)):
  df = dfs[i]
  name = df.name
#   print(i, name)
  df['sqrt/auc'] = (df['auc'] * df['click/time_auc']) ** 0.5
  df['group/sqrt/auc'] =  (df['group/auc'] * df['group/click/time_auc']) ** 0.5
  try:
    df['rmse'] = 1. - df['mse'] ** 0.5
    df['click/rmse'] = 1. - df['click/mse'] ** 0.5
    df['Dur/click/rmse'] = 1. - df['Dur/click/mse'] ** 0.5
  except Exception:
    pass
  dfs[i] = df
  m[name] = df
  
models = m.keys()

# models = [
#         'dlrm2', 'baseline', 'baseline.off'
#          ]

# print(models)

def is_ok(key):
#   if 'baseline' in key or 'trans' in key or key == 'dlrm' or key == 'concat':
#     return True
#   return False
  return True

dfs_ = [m[key][m[key].hour>=2020050100] for key in models if key in m.keys() if is_ok(key)]

# x = ['fm.finetune', 'old']
# dfs_ = [m[key] for key in x] + [m[key] for key in models if key not in x]
# # print(m.keys())

# group/auc is gauc using impression as weight, group/auc2 not using weight, just mean auc with / num_users
# so group/auc2 might more realted to read_ratio as it treat each user equally, hight group/auc2 means better
# performance on users with less impressions(and views)
metric_names = [
  'stats/num_instances',  
  'Click/group/auc',
#   'Click/group/auc2',
  'Dur/group/click/time_auc',
  'group/auc',
  'group/click/time_auc',
  'Click/cold/group/auc'
#   'Dur/group/click/ndcg_dur',
#   'Dur/click/rmse',
#   'group/sqrt/auc',
#    'group/auc2', 'cold/group/auc', 'cold/group/auc2', 'quality/group/auc', 'quality/group/auc2',
#   'group/click/time_auc', 'cold/group/click/time_auc', 'quality/group/click/time_auc', 'click/rmse', 'rmse',
#   #   'sqrt/auc', 'auc', 'clickmids/auc', 'cold/auc', 'quality/auc',  
#   #   'click/time_auc', 'cold/click/time_auc', 'quality/click/time_auc',
#   'rcr_auc',
#   'ptime_auc',
# #   'Dur/click/time_auc',
#   'loss/click',
#   'loss/dur',
#   'inv_rate',
#   'stats/num_instances',
]
# metric_names += [x for x in dfs_[0].columns if 'loss' in x or 'mse' in x or 'mae' in x]
# show(dfs_, metric_names, smoothing=0.5, limit=168, exclude='Click,Dur')

focus = True # focus means focus by the first model
# focus = False
# focus_name = 'transformer'
focus_name = 'baseline'
smoothing= 0.5
line(dfs_, x='hour', y=metric_names, smoothing=smoothing, focus=focus, focus_name=focus_name)

In [ ]:
metrics = [
  'group/top1_click', #precision@1
  'group/top3_click', #precision@3
  'group/top1_dur', #top1 position mean duration
  'group/top3_dur', #top3 position mean duration
  'group/click/top1_dur', 
  'group/click/top3_dur',
]
line(dfs_, 'hour', metrics, smoothing=smoothing, focus=focus, focus_name=focus_name)

In [ ]:
metrics = [
  'group/first_click_position',
  'group/last_click_position',
  'group/ndcg_click',   # ndcg@all for click
  'group/ndcg3_click',  # ndcg@3 for click (using 0-1 label)
  'group/ndcg3_dur',    # ndcg@3 using duration as label
  'group/ndcg_dur',
#   'group/ndcg7_click',
#   'group/ndcg14_click',
  'group/click/ndcg_dur',
  'group/click/ndcg3_dur',
#   'group/click/ndcg7_dur',
#   'group/click/ndcg14_dur',
  'quality/group/ndcg_dur',
  'quality/group/click/ndcg_dur',
  'quality/group/click/ndcg3_dur',
]
line(dfs_, 'hour', metrics, smoothing=smoothing, focus=focus, focus_name=focus_name)

In [ ]:
# metrics = [
#   'duration',
#   'perf/total_time',
#   'perf/train_time',
#   'perf/valid_time',
# ]
# dfs_ = [m[key] for key in models if key in m.keys() if is_ok(key) and 'records' not in key]
# line(dfs_, 'hour', metrics, smoothing=smoothing, focus=focus, focus_name=focus_name)